<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/GEMINI3_FLASH_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

import os
from google import genai
from google.genai import types

from warnings import filterwarnings
filterwarnings('ignore')

#

# 2. Securely fetch your API Key from Colab Secrets
try:
    from google.colab import userdata
    GEMINI_API_KEY = userdata.get('GEMINI')
except (ImportError, KeyError):
    # Fallback for local environments
    GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY")

# 3. Initialize the Client
client = genai.Client(api_key=GEMINI_API_KEY)

# Use the preview model ID for Gemini 3 Flash
MODEL_ID = "gemini-3-flash-preview"

def demo_gemini_3_flash(prompt, thinking_level="medium"):
    """
    thinking_level: "minimal", "low", "medium", or "high" (Default: high)
    """
    print(f"\n--- Model: {MODEL_ID} | Level: {thinking_level.upper()} ---")

    config = types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(
            thinking_level=thinking_level,
            include_thoughts=True  # Returns the model's internal reasoning
        )
    )

    response = client.models.generate_content(
        model=MODEL_ID,
        contents=prompt,
        config=config
    )

    # 4. Parsing the Response (Thinking models return multiple parts)
    for part in response.candidates[0].content.parts:
        if part.thought:
            print(f"🧠 THOUGHTS:\n{part.text}\n")
        else:
            print(f"✅ RESPONSE:\n{part.text}")

# --- Test Case ---
prompt = "Explain how to optimize a nested loop in Python for a dataset of 1 million items."
demo_gemini_3_flash(prompt, thinking_level="medium")


--- Model: gemini-3-flash-preview | Level: MEDIUM ---
🧠 THOUGHTS:
**Tackling the Beast: Optimizing a Million-Item Nested Loop in Python**

Okay, so I've got a problem. A million items and nested loops – that spells out *disaster* in Python land. We're talking $O(n^2)$, which is a colossal $10^{12}$ operations. That's simply not feasible in standard Python without waiting an eternity. So, how do I conquer this complexity? Let's break it down.

First, I need to get my head around *why* Python's nested loops are so slow. It's that darn interpreter overhead, dynamic typing, and of course, that killer quadratic complexity. Now, I have a game plan for optimization.

The core of it is to find a way to *not* use nested loops, or reduce the operations that they perform. I need to be aiming for $O(n)$ or at worst, $O(n \log n)$. If it's something like finding duplicates or matching pairs, using a set or dictionary lookup is the obvious first move.

Next, vectorization with NumPy is the way to g

## AOCC

In [1]:
import os
from google import genai
from google.genai import types

# 2. Get API Key from Colab Secrets
try:
    from google.colab import userdata
    GEMINI_API_KEY = userdata.get('GEMINI')
except (ImportError, KeyError):
    print("Error: Ensure a secret named 'GEMINI' is set in your Colab Secrets.")
    GEMINI_API_KEY = None

client = genai.Client(api_key=GEMINI_API_KEY)
MODEL_ID = "gemini-3-flash-preview"

# --- THE OCC AGENTIC FACTORY ---

def call_occ_agent(agent_name, role_instruction, task, reasoning_level):
    """
    Standardizes agent calls with specific reasoning levels.
    """
    config = types.GenerateContentConfig(
        system_instruction=role_instruction,
        thinking_config=types.ThinkingConfig(
            thinking_level=reasoning_level, # "low", "medium", or "high"
            include_thoughts=True
        )
    )

    response = client.models.generate_content(
        model=MODEL_ID,
        contents=task,
        config=config
    )

    # Return structured data for the orchestrator to handle
    return {
        "name": agent_name,
        "thought": next((p.text for p in response.candidates[0].content.parts if p.thought), "None"),
        "command": next((p.text for p in response.candidates[0].content.parts if not p.thought), "No command issued.")
    }

# --- OCC MULTI-AGENT ORCHESTRATOR ---

def orchestrate_occ_response(crisis_summary):
    print(f"🚨 ALERT: {crisis_summary}")

    # 1. THE DIRECTOR (HIGH REASONING)
    # Tasks: Strategize, decompose crisis, and assign workers.
    director = call_occ_agent(
        "OCC Director",
        "You are the Director of the OCC. Break this crisis into sub-tasks for Fleet and Crew agents.",
        crisis_summary,
        "high"
    )
    print(f"\n🏗️ [DIRECTOR PLAN]:\n{director['command']}")

    # 2. FLEET OPS (LOW REASONING)
    # Tasks: Fast, high-throughput re-routing of aircraft.
    fleet_agent = call_occ_agent(
        "Fleet Ops",
        "You handle tail assignments and fuel routing. Focus on speed.",
        f"Director's Plan: {director['command']}. Re-route flights now.",
        "low"
    )
    print(f"\n✈️ [FLEET COMMAND]:\n{fleet_agent['command']}")

    # 3. CREW LEGAL (MEDIUM REASONING)
    # Tasks: Logic-based compliance checks for pilot duty hours.
    crew_agent = call_occ_agent(
        "Crew Legal",
        "You verify FAA rest hours and crew legality. Use logic.",
        f"Director's Plan: {director['command']}. Check if current crew can handle the delay.",
        "medium"
    )
    print(f"\n⚖️ [CREW COMPLIANCE]:\n{crew_agent['command']}")

    # 4. FINAL CONSOLIDATION (HIGH REASONING)
    # Tasks: Review all worker outputs and issue the final executive order.
    final_task = f"Director Strategy: {director['command']}\nFleet Plan: {fleet_agent['command']}\nCrew Status: {crew_agent['command']}"
    final_output = call_occ_agent(
        "Final Dispatch",
        "Finalize the OCC action plan. Be authoritative.",
        final_task,
        "high"
    )

    print(f"\n🏁 [FINAL OCC EXECUTIVE ORDER]:\n{final_output['command']}")

# --- EXECUTE ---
crisis = "Power outage at SFO. 12 flights inbound, 3 on low fuel. Divert to SJC or OAK."
orchestrate_occ_response(crisis)

🚨 ALERT: Power outage at SFO. 12 flights inbound, 3 on low fuel. Divert to SJC or OAK.

🏗️ [DIRECTOR PLAN]:
This is the Director of the OCC. We have a Code Red situation at SFO. Our primary objective is the safe recovery of all 12 aircraft, with absolute priority given to the three low-fuel emergencies. SJC and OAK are our primary diversion points.

Effective immediately, I am activating the following task list for Fleet and Crew agents:

### **Fleet Agent Tasks**

**1. Emergency Diversion Sequencing (Immediate)**
*   **Identify:** Pinpoint the tail numbers of the 3 low-fuel flights. 
*   **Coordinate:** Contact SJC and OAK ATC towers immediately to declare emergency status for these 3 tails and secure priority landing slots. 
*   **Assign:** Route 2 low-fuel flights to SJC and 1 to OAK to balance the immediate taxi-way load.

**2. Slot & Gate Management (T+15 Minutes)**
*   **Negotiate:** Contact SJC and OAK Station Managers to secure gate or hardstand assignments for the remaining 9 

## Stateful OCC Orchestration with Thought Signatures

In [3]:
# 1. Setup
!pip install -U -q 'google-genai>=1.51.0'

In [5]:
import os
from google import genai
from google.genai import types

# 2. Key Fetch
try:
    from google.colab import userdata
    GEMINI_API_KEY = userdata.get('GEMINI')
except (ImportError, KeyError):
    GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY")

client = genai.Client(api_key=GEMINI_API_KEY)
MODEL_ID = "gemini-3-flash-preview"

# --- FIXED STATEFUL AGENT WRAPPER ---
def run_stateful_agent(role, task, level, history=None):
    """
    Fixed: Uses keyword argument 'text=' in Part.from_text.
    """
    config = types.GenerateContentConfig(
        system_instruction=f"You are the {role}. Execute with precision.",
        thinking_config=types.ThinkingConfig(
            thinking_level=level,
            include_thoughts=True
        )
    )

    # If no history exists, start fresh; otherwise, append the task
    contents = history or []

    # FIX: Explicitly use text=task
    contents.append(types.Content(
        role="user",
        parts=[types.Part.from_text(text=task)]
    ))

    response = client.models.generate_content(
        model=MODEL_ID,
        contents=contents,
        config=config
    )

    # Append the entire response content to history to preserve Thought Signatures
    model_content = response.candidates[0].content
    contents.append(model_content)

    return response, contents

# --- MULTI-TURN OCC RECOVERY ---

# TURN 1: The Director (HIGH) - Strategic Decision
crisis = "Main power bus at SFO failed. 12 inbounds, 3 emergency fuel. Create diversion plan."
resp_dir, occ_history = run_stateful_agent("OCC Director", crisis, "high")

print(f"🏗️ DIRECTOR PLAN (Turn 1):\n{resp_dir.text}\n")

# TURN 2: Fleet Agent (LOW) - Tactical Execution
# The Fleet Agent 'low' reasoning now inherits context from the Director 'high' reasoning.
fleet_task = "List the tail-by-tail assignments for the emergency fuel flights only."
resp_fleet, occ_history = run_stateful_agent("Fleet Agent", fleet_task, "low", history=occ_history)

print(f"✈️ FLEET COMMAND (Turn 2):\n{resp_fleet.text}")

🏗️ DIRECTOR PLAN (Turn 1):
**TO:** All Stations / NorCal TRACON / SFO Ground Ops
**FROM:** OCC Director
**SUBJECT:** EMERGENCY DIVERSION PLAN – SFO MAIN POWER BUS FAILURE
**STATUS:** URGENT – CRITICAL OPS

SFO is currently **ATC ZERO** for arrivals due to primary power failure. All inbound traffic must divert immediately.

### **I. PRIORITY 1: EMERGENCY FUEL AIRCRAFT (3)**
*These aircraft have immediate priority for the shortest vector and landing clearance.*

1.  **ACFT 1 (Heavy/Long-Haul):** Divert to **SJC (San Jose)**. 
    *   *Reasoning:* Longest runway availability and immediate heavy-gate clearance.
2.  **ACFT 2 (Narrowbody):** Divert to **OAK (Oakland)**. 
    *   *Reasoning:* Minimum flight time from SFO approach path; high-speed taxiway availability.
3.  **ACFT 3 (Narrowbody):** Divert to **OAK (Oakland)**.
    *   *Reasoning:* Parallel runway ops at OAK allow for simultaneous emergency handling if required.

### **II. PRIORITY 2: REMAINING INBOUNDS (9)**
*Distribution to av

## AOC FINAL

In [6]:


import os
from google import genai
from google.genai import types

# 2. AUTHENTICATION & CLIENT INITIALIZATION
try:
    from google.colab import userdata
    GEMINI_API_KEY = userdata.get('GEMINI')
except (ImportError, KeyError):
    # Fallback for local environments
    GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY")

if not GEMINI_API_KEY:
    raise ValueError("API Key 'GEMINI' not found in Colab Secrets.")

client = genai.Client(api_key=GEMINI_API_KEY)
MODEL_ID = "gemini-3-flash-preview"

# --- CORE ORCHESTRATION ENGINE ---

def run_occ_agent(role, task, level, history=None):
    """
    Executes a specific agent role at a specific thinking level (LOW/MEDIUM/HIGH).
    Correctly handles 'text=' keyword argument and preserves Thought Signatures.
    """
    config = types.GenerateContentConfig(
        system_instruction=f"You are the {role}. Provide clear, actionable, operational commands.",
        thinking_config=types.ThinkingConfig(
            thinking_level=level,
            include_thoughts=True
        ),
        temperature=1.0 # Optimized for Gemini 3
    )

    # Manage stateful history
    contents = history or []

    # Correct Part.from_text syntax for google-genai SDK
    contents.append(types.Content(
        role="user",
        parts=[types.Part.from_text(text=task)]
    ))

    # API Call
    response = client.models.generate_content(
        model=MODEL_ID,
        contents=contents,
        config=config
    )

    # Append the response (including thoughts/signatures) back into history
    model_output = response.candidates[0].content
    contents.append(model_output)

    return response, contents

# --- THE FULL OCC WORKFLOW ---

def execute_full_occ_recovery():
    print("🚀 INITIALIZING MULTI-AGENT OCC ORCHESTRATION...")

    # TURN 1: THE DIRECTOR (HIGH REASONING)
    # Responsibility: High-level strategy and crisis decomposition.
    crisis = "SFO Main Power Bus failure. 12 flights inbound, 3 emergency fuel. Create diversion strategy."
    res_dir, history = run_occ_agent("OCC Director", crisis, "high")

    print(f"\n🏗️ [DIRECTOR - HIGH]:\n{res_dir.text}")
    print(f"{'='*80}")

    # TURN 2: FLEET AGENT (LOW REASONING)
    # Responsibility: Fast, tactical tail assignments based on Director's plan.
    fleet_task = "List the specific tail numbers and ETA logic for the 3 emergency fuel flights mentioned by the Director."
    res_fleet, history = run_occ_agent("Fleet Agent", fleet_task, "low", history=history)

    print(f"\n✈️ [FLEET - LOW]:\n{res_fleet.text}")
    print(f"{'='*80}")

    # TURN 3: CREW LEGAL (MEDIUM REASONING)
    # Responsibility: Constraint checking. Apply FAA Part 117 rest-hour logic.
    crew_task = "Analyze the 3 emergency tails. If crews started at 06:00, will they time out after landing? Identify grounding risks."
    res_crew, history = run_occ_agent("Crew Agent", crew_task, "medium", history=history)

    print(f"\n⚖️ [CREW - MEDIUM]:\n{res_crew.text}")
    print(f"{'='*80}")

    # TURN 4: FINAL CONSOLIDATION (HIGH REASONING)
    # Responsibility: Audit workers and issue final executive order.
    final_task = "Audit the Fleet routing and Crew compliance. Issue a final Execute Order for the next 4 hours."
    res_final, history = run_occ_agent("Director Final", final_task, "high", history=history)

    print(f"\n🏁 [FINAL EXECUTIVE ORDER - HIGH]:\n{res_final.text}")

# RUN THE SYSTEM
execute_full_occ_recovery()

🚀 INITIALIZING MULTI-AGENT OCC ORCHESTRATION...

🏗️ [DIRECTOR - HIGH]:
**TO:** SFO TRACON, SFO Tower, All Inbound Flight Decks, Station Managers (OAK, SJC, SMF)
**FROM:** OCC Director
**SUBJECT:** OPERATIONAL COMMAND – SFO MAIN POWER BUS FAILURE – DIVERSION PROTOCOL

SFO is closed to all arrivals effective immediately. Power bus failure has compromised runway lighting and ground handling systems. 

**I. IMMEDIATE PRIORITY: EMERGENCY FUEL STATUS (3 AIRCRAFT)**
*   **COMMAND:** Divert Emergency Fuel aircraft to **OAK (Oakland)** and **SJC (San Jose)** immediately.
*   **ALLOCATION:** 
    *   **EF Flight 1:** Clear for immediate approach OAK RWY 12/30.
    *   **EF Flight 2:** Clear for immediate approach SJC RWY 12L/30R.
    *   **EF Flight 3:** Clear for immediate approach OAK RWY 12/30.
*   **INSTRUCTION:** SFO Tower to hand off directly to OAK/SJC Approach with "Emergency Priority" status. Ground handling at OAK/SJC: Clear gates 1-4 for immediate engine-off deplaning.

**II. SECONDAR